In [9]:
! pip install pandas

     |████████████████████████████████| 11.7 MB 10.9 MB/s eta 0:00:01
     |████████████████████████████████| 503 kB 8.5 MB/s eta 0:00:01
     |████████████████████████████████| 16.8 MB 9.0 MB/s eta 0:00:011


In [1]:
import pyspark
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

In [31]:
!wget https://nyc-tlc.s3.amazonaws.com/trip+data/fhvhv_tripdata_2021-02.csv

--2022-03-02 21:42:00--  https://nyc-tlc.s3.amazonaws.com/trip+data/fhvhv_tripdata_2021-02.csv
Resolving nyc-tlc.s3.amazonaws.com (nyc-tlc.s3.amazonaws.com)... 52.217.230.177
Connecting to nyc-tlc.s3.amazonaws.com (nyc-tlc.s3.amazonaws.com)|52.217.230.177|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 733822658 (700M) [text/csv]
Saving to: ‘fhvhv_tripdata_2021-02.csv’

fhvhv_tripdata_2021 100%[===================>] 699.83M  7.03MB/s    in 4m 22s  

2022-03-02 21:46:23 (2.67 MB/s) - ‘fhvhv_tripdata_2021-02.csv’ saved [733822658/733822658]



In [32]:
!wc -l fhvhv_tripdata_2021-02.csv

11613943 fhvhv_tripdata_2021-02.csv


Integer - 4 bytes
Long - 8 bytes

In [3]:
from pyspark.sql import types

In [4]:
schema = types.StructType([
    types.StructField('hvfhs_license_num', types.StringType(), True),
    types.StructField('dispatching_base_num', types.StringType(), True),
    types.StructField('pickup_datetime', types.TimestampType(), True),
    types.StructField('dropoff_datetime', types.TimestampType(), True),
    types.StructField('PULocationID', types.IntegerType(), True),
    types.StructField('DOLocationID', types.IntegerType(), True),
    types.StructField('SR_Flag', types.StringType(), True)
])

In [7]:
df = spark.read \
    .option("header", "true") \
    .schema(schema) \
    .csv('/mnt/c/Jupyter_wsl/fhvhv_tripdata_2021-02.csv')

Question 2

In [49]:
df = df.repartition(24)

In [43]:
df.write.parquet('fhvhv/2021/03/')

In [8]:
#Какой порт использует
spark.sparkContext.uiWebUrl

'http://172.19.10.61:4041'

In [9]:
df.show()

+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|hvfhs_license_num|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|
+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|           HV0003|              B02764|2021-02-01 00:10:40|2021-02-01 00:21:09|          35|          39|   null|
|           HV0003|              B02764|2021-02-01 00:27:23|2021-02-01 00:44:01|          39|          35|   null|
|           HV0005|              B02510|2021-02-01 00:28:38|2021-02-01 00:38:27|          39|          91|   null|
|           HV0005|              B02510|2021-02-01 00:43:37|2021-02-01 01:23:20|          91|         228|   null|
|           HV0003|              B02872|2021-02-01 00:08:42|2021-02-01 00:17:57|         126|         250|   null|
|           HV0003|              B02872|2021-02-01 00:26:02|2021-02-01 00:42:51|

Question 3

In [10]:
from pyspark.sql import functions as F

In [11]:
df.filter(F.to_date(df.pickup_datetime) == '2021-02-15').count()

367170

Question 4

In [12]:
diff_secs_col = df.dropoff_datetime.cast("long") - df.pickup_datetime.cast("long")

In [13]:
df1 = df.withColumn( "diff_mins", (df.dropoff_datetime.cast("long") - df.pickup_datetime.cast("long"))/60)

In [14]:
df1.orderBy(F.desc("diff_mins")).show(5)

+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+-----------------+
|hvfhs_license_num|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|        diff_mins|
+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+-----------------+
|           HV0005|              B02510|2021-02-11 13:40:44|2021-02-12 10:39:44|         247|          41|   null|           1259.0|
|           HV0004|              B02800|2021-02-17 15:54:53|2021-02-18 07:48:34|         242|         254|   null|953.6833333333333|
|           HV0004|              B02800|2021-02-20 12:08:15|2021-02-21 00:22:14|         188|          55|   null|733.9833333333333|
|           HV0003|              B02864|2021-02-03 20:24:25|2021-02-04 07:41:58|          51|         147|   null|           677.55|
|           HV0003|              B02887|2021-02-19 23:17:44|2021-02-2

In [21]:
df.select('dispatching_base_num').groupBy('dispatching_base_num').count().orderBy(F.desc("count")).show(5)

+--------------------+-------+
|dispatching_base_num|  count|
+--------------------+-------+
|              B02510|3233664|
|              B02764| 965568|
|              B02872| 882689|
|              B02875| 685390|
|              B02765| 559768|
+--------------------+-------+
only showing top 5 rows



Question 6

In [23]:
df_zones = spark.read.parquet('/mnt/c/Jupyter_wsl/zones/')

In [28]:
df_zones.show()

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
|         6|Staten Island|Arrochar/Fort Wad...|   Boro Zone|
|         7|       Queens|             Astoria|   Boro Zone|
|         8|       Queens|        Astoria Park|   Boro Zone|
|         9|       Queens|          Auburndale|   Boro Zone|
|        10|       Queens|        Baisley Park|   Boro Zone|
|        11|     Brooklyn|          Bath Beach|   Boro Zone|
|        12|    Manhattan|        Battery Park| Yellow Zone|
|        13|    Manhattan|   Battery Park City| Yellow Zone|
|        14|     Brookly

In [26]:
df.registerTempTable('trips')

/home/olia/.local/lib/python3.8/site-packages/pyspark/sql/dataframe.py:138: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn(


In [27]:
df_zones.registerTempTable('zones')

/home/olia/.local/lib/python3.8/site-packages/pyspark/sql/dataframe.py:138: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn(


In [33]:
df_lo = spark.sql("""
SELECT
    pu.Zone as pu_zone,
    do.Zone as do_zone,
    count(1) as lo_cnt
FROM
    trips
    left join zones pu
      on pu.LocationID = trips.PULocationID
    left join zones do
      on do.LocationID = trips.DOLocationID
GROUP BY 
    pu.Zone,
    do.Zone
""")

In [36]:
df_lo.orderBy(F.desc("lo_cnt")).show(5)

+-------------------+-------------------+------+
|            pu_zone|            do_zone|lo_cnt|
+-------------------+-------------------+------+
|      East New York|      East New York| 45041|
|       Borough Park|       Borough Park| 37329|
|           Canarsie|           Canarsie| 28026|
|Crown Heights North|Crown Heights North| 25976|
|          Bay Ridge|          Bay Ridge| 17934|
+-------------------+-------------------+------+
only showing top 5 rows

